In [33]:
from google.colab import files
uploaded = files.upload()

Saving cleaned_burnout_dataset.csv to cleaned_burnout_dataset (2).csv


In [34]:
import pandas as pd
df = pd.read_csv("cleaned_burnout_dataset.csv")
df.head()


,EmployeeID,Age,Gender,Country,JobRole,Department,YearsAtCompany,WorkHoursPerWeek,RemoteWork,BurnoutLevel,...,CommuteTime,HasMentalHealthSupport,ManagerSupportScore,HasTherapyAccess,MentalHealthDaysOff,SalaryRange,WorkLifeBalanceScore,TeamSize,CareerGrowthScore,BurnoutRisk
0,1001,50,Male,UK,Sales Associate,HR,14,47,No,3.37,...,117,No,3.15,Yes,8,40K-60K,8.82,6,9.20,0
1,1002,36,Male,Germany,Software Engineer,IT,1,59,Hybrid,7.39,...,8,Yes,4.40,Yes,4,80K-100K,2.80,45,8.46,1
2,1003,29,Non-binary,India,IT Admin,IT,13,59,Hybrid,7.10,...,75,No,3.63,No,6,80K-100K,7.28,7,7.96,1
3,1004,42,Male,Australia,HR Specialist,IT,15,31,Yes,4.18,...,43,Yes,4.50,Yes,9,60K-80K,1.31,11,8.90,0
4,1005,40,Male,Brazil,Customer Support,Support,6,34,Yes,8.28,...,58,Yes,5.51,Yes,6,<40K,1.17,18,8.88,1


In [35]:
from sklearn.preprocessing import StandardScaler

# Step 1: Select numeric columns
numeric_cols = ['SleepHours', 'WorkHoursPerWeek', 'BurnoutLevel', 'MentalHealthDaysOff']

# Step 2: Initialize scaler
scaler = StandardScaler()

# Step 3: Fit and transform
df_scaled = df.copy()
df_scaled[numeric_cols] = scaler.fit_transform(df[numeric_cols])

# Step 4: Check results
df_scaled[numeric_cols].head()


,SleepHours,WorkHoursPerWeek,BurnoutLevel,MentalHealthDaysOff
0,0.319776,0.293989,-0.831171,1.210845
1,0.458507,1.707398,0.730817,-0.190866
2,-0.928806,1.707398,0.618137,0.509990
3,1.498992,-1.590556,-0.516442,1.561273
4,-1.622463,-1.237204,1.076631,0.509990


In [36]:
from sklearn.preprocessing import OneHotEncoder

# Encode all categorical (text) columns using OneHotEncoder
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
encoder = OneHotEncoder(sparse_output=False, drop='first')
encoded_values = encoder.fit_transform(df[cat_cols])
encoded_names = encoder.get_feature_names_out(cat_cols)
encoded_df = pd.DataFrame(encoded_values, columns=encoded_names)

# Combine encoded categorical data with scaled numeric features
final_df = pd.concat([df_scaled[numeric_cols], encoded_df], axis=1)

# Add target column back for modeling
encoded_df['StressLevel'] = df['StressLevel']

# Show the first few rows of the final dataset
final_df.head()

,SleepHours,WorkHoursPerWeek,BurnoutLevel,MentalHealthDaysOff,Gender_Male,Gender_Non-binary,Gender_Prefer not to say,Country_Brazil,Country_Canada,Country_Germany,...,Department_Sales,Department_Support,RemoteWork_No,RemoteWork_Yes,HasMentalHealthSupport_Yes,HasTherapyAccess_Yes,SalaryRange_40K-60K,SalaryRange_60K-80K,SalaryRange_80K-100K,SalaryRange_<40K
0,0.319776,0.293989,-0.831171,1.210845,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,0.458507,1.707398,0.730817,-0.190866,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
2,-0.928806,1.707398,0.618137,0.509990,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.498992,-1.590556,-0.516442,1.561273,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0
4,-1.622463,-1.237204,1.076631,0.509990,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0


In [37]:
from sklearn.preprocessing import StandardScaler

#Select only numeric columns from the encoded data
numeric_columns = encoded_df.select_dtypes(include=['int64', 'float64']).columns

# Create a scaler object
scaler = StandardScaler()

#Fit and transform the numeric columns
scaled_numeric = scaler.fit_transform(encoded_df[numeric_columns])

#Replace the original numeric columns with the scaled ones
encoded_df[numeric_columns] = scaled_numeric

#The data is now normalized and ready for model training
print("Scaled numerical features:\n")
print(encoded_df.head())


Scaled numerical features:

   Gender_Male  Gender_Non-binary  Gender_Prefer not to say  Country_Brazil  \
0     1.728977          -0.580943                 -0.580943       -0.416233   
1     1.728977          -0.580943                 -0.580943       -0.416233   
2    -0.578377           1.721340                 -0.580943       -0.416233   
3     1.728977          -0.580943                 -0.580943       -0.416233   
4     1.728977          -0.580943                 -0.580943        2.402501   

   Country_Canada  Country_Germany  Country_India  Country_UK  Country_USA  \
0        -0.41126        -0.395620      -0.427744    2.454743    -0.396746   
1        -0.41126         2.527679      -0.427744   -0.407375    -0.396746   
2        -0.41126        -0.395620       2.337845   -0.407375    -0.396746   
3        -0.41126        -0.395620      -0.427744   -0.407375    -0.396746   
4        -0.41126        -0.395620      -0.427744   -0.407375    -0.396746   

   JobRole_Data Scientist  .

In [38]:
import numpy as np

# Check correlation with the target
correlation = encoded_df.corr(numeric_only=True)['StressLevel'].sort_values(ascending=False)

# Print top features correlated with StressLevel (excluding StressLevel itself)
print("Top correlated features with StressLevel:\n")
print(correlation.drop('StressLevel').head(10))


Top correlated features with StressLevel:

Department_IT                0.030686
Country_USA                  0.029805
SalaryRange_60K-80K          0.029182
JobRole_Project Manager      0.028002
HasTherapyAccess_Yes         0.023196
JobRole_Sales Associate      0.020579
Country_UK                   0.017729
SalaryRange_40K-60K          0.015369
Department_Marketing         0.014847
JobRole_Marketing Manager    0.011900
Name: StressLevel, dtype: float64


In [39]:
from sklearn.feature_selection import mutual_info_regression

# Separate features and target
X = encoded_df.drop('StressLevel', axis=1)
y = encoded_df['StressLevel']

# Calculate mutual information scores
mi_scores = mutual_info_regression(X, y)

# Convert to a nice Pandas Series for viewing
mi_scores_series = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)

# Show top 10 most informative features
print("\nTop features using mutual information:\n")
print(mi_scores_series.head(10))



Top features using mutual information:

Department_IT           0.021821
Country_Brazil          0.009064
JobRole_IT Admin        0.008256
SalaryRange_40K-60K     0.007968
HasTherapyAccess_Yes    0.006939
SalaryRange_80K-100K    0.006865
Country_India           0.006308
SalaryRange_<40K        0.005829
RemoteWork_Yes          0.005262
Department_Support      0.004877
dtype: float64


In [40]:
# Let's create two interaction features

# 1. Interaction between Work Hours and Stress Level
encoded_df['Work_Stress'] = df['WorkHoursPerWeek'] * df['StressLevel']

# 2. Interaction between Sleep Hours and Stress Level
encoded_df['Sleep_Stress'] = df['SleepHours'] * df['StressLevel']


In [41]:
from sklearn.feature_selection import mutual_info_regression

# Separate features and target
X = encoded_df.drop('StressLevel', axis=1)
y = encoded_df['StressLevel']

# Calculate mutual information scores
mi_scores = mutual_info_regression(X, y)

# Make it readable
import pandas as pd
mi_scores_series = pd.Series(mi_scores, index=X.columns).sort_values(ascending=False)

# Show top 10 important features
print("Top 10 features using Mutual Information:")
print(mi_scores_series.head(10))


Top 10 features using Mutual Information:
Work_Stress                  1.255951
Sleep_Stress                 1.124483
Country_Germany              0.022313
Department_IT                0.016728
JobRole_Software Engineer    0.014046
SalaryRange_40K-60K          0.012914
Country_Brazil               0.009038
JobRole_HR Specialist        0.007586
Gender_Male                  0.005362
Department_Marketing         0.005122
dtype: float64


In [42]:
#Get top 10 feature names based on mutual info scores
top_features = mi_scores_series.head(10).index.tolist()

print("Selected top features for modeling:")
print(top_features)


Selected top features for modeling:
['Work_Stress', 'Sleep_Stress', 'Country_Germany', 'Department_IT', 'JobRole_Software Engineer', 'SalaryRange_40K-60K', 'Country_Brazil', 'JobRole_HR Specialist', 'Gender_Male', 'Department_Marketing']


In [43]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

#Features and target
X = encoded_df[top_features]
y = encoded_df['StressLevel']

#Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Initialize models
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso()
}

#Train & evaluate
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"{name}")
    print(f"Mean Squared Error: {mse:.2f}")
    print(f"R² Score: {r2:.2f}")
    print("")

Linear Regression
Mean Squared Error: 0.10
R² Score: 0.89

Ridge Regression
Mean Squared Error: 0.10
R² Score: 0.89

Lasso Regression
Mean Squared Error: 0.11
R² Score: 0.89



Among Linear, Ridge, and Lasso regression, the model with the best R² and lowest MSE performed best.
here R² is same for all three so lets check mean squared error. we can see that linear and ridge are best for this data set with value 0.10